In [ ]:
# Loading the train and test dataset
import pandas as pd
df2_train = pd.read_csv("training_task_b_source_tweets.csv")
df2_test = pd.read_csv("test_task_b_tweets.csv")

In [ ]:
# Importing the necessary libraries

import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
import nltk
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
nltk.download('punkt')
nltk.download('wordnet')

# converting the text into lowercase
df2_train["text"]=df2_train["text"].str.lower()
df2_test["text"]=df2_test["text"].str.lower()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# applying the necessary preprocessing such as removing stopwords,stemming,lemmatizing

def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)
def stop_words(text):
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  return text
def getStemmText(text):
 tokens=word_tokenize(text)
 ps = PorterStemmer()
 tokens=[ps.stem(word) for word in tokens]
 return ' '.join(tokens)

In [ ]:
df2_train['text'] = list(map(stop_words,df2_train['text']))
df2_train['text'] = list(map(getLemmText,df2_train['text']))
df2_train['text'] = list(map(getStemmText,df2_train['text']))

df2_test['text'] = list(map(stop_words,df2_test['text']))
df2_test['text'] = list(map(getLemmText,df2_test['text']))
df2_test['text'] = list(map(getStemmText,df2_test['text']))

In [ ]:
# dropping the columns which are not being used for model building
df2_train.drop(df2_train.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)
df2_test.drop(df2_test.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)
# converting the target variable into one hot encoding format which is benficial in later stages
map_dict =  {'true':0,'false':1, 'unverified':2}
df2_train['label'] = df2_train.label.map(map_dict)
df2_test['label'] = df2_test.label.map(map_dict)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(df2_train.label)
y_test = to_categorical(df2_test.label)

In [ ]:
!pip install transformers
import transformers

     |████████████████████████████████| 4.2 MB 12.3 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 6.6 MB 68.8 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# importing the pretrained BERT model "bert-base-cased" which is popular for sentiment analysis
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# converting the input text into tokenized vector format inorder to feed into ML algo.Here we have defined few parameters such as max_length,padding
x_train = tokenizer(
    text=df2_train.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    padding="max_length", 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

x_test = tokenizer(
    text=df2_test.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    padding="max_length", 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
# Defining the adam optimizer as part of the model architecture

optimizer = Adam(
    learning_rate=5e-05,  
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=36
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


9/9 [==============================] - 200s 20s/step - loss: 1.1252 - balanced_accuracy: 0.4141 - val_loss: 1.0809 - val_balanced_accuracy: 0.3750


In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df2_test.label

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print(f1_score(y_true, y_predicted , average="macro"))

0.18181818181818185


In [ ]:
from sklearn.metrics import mean_squared_error
import math
y_true = [np.argmax(val) for val in y_true]
conf = []
for i in range(len(y_true)):
  conf.append(max(predicted_raw[i]))
RMSE = mean_squared_error([1]*len(conf), conf,squared = False)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.6067886974007771
